In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
import pandas

train = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ud_paths_v0.0.5.csv')
test = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ud_paths_v0.0.5.csv')
    

In [3]:
relation_to_paths = train.groupby(['relation'])['path'].unique().to_dict()

In [4]:
def label_sentence_recall(test_row, relation):
    path = test_row['path']
    true_relation = test_row['relation']
    
    if path in relation_to_paths[relation]:
        if true_relation == relation:
            return 'TP'
        else:
            return 'FP'
    else:
        return 'N'

In [5]:
for relation in filter(lambda relation: relation != 'no_relation', relation_to_paths.keys()):
    test[relation] = test.apply(lambda row : label_sentence_recall(row, relation), axis=1)    

In [8]:
stats = []
overall_tp = 0
overall_fp = 0

for relation in filter(lambda relation: relation != 'no_relation', relation_to_paths.keys()):

    counts = test[relation].value_counts().to_dict()
        
    tp = counts.get('TP', 0)
    fp = counts.get('FP', 0)
    precision = tp / (tp+fp)

    stats.append( (relation, round(precision,4) ) )
    
    overall_tp += tp
    overall_fp += fp
    
overall_precision = overall_tp / (overall_tp+overall_fp)


stats.append( ('overall', round(overall_precision,4) ) )
stats = sorted(stats, key=lambda x: x[1], reverse=True)

In [12]:
import plotly.graph_objs as go


overall_index = next(i for (i, stat) in enumerate(stats) if stat[0] == 'overall')

colors = ['rgb(58,200,225)'] * len(stats)
colors[overall_index] = 'rgb(181,59,89)'


trace = go.Bar(
    x=[stat[0] for stat in stats],
    y=[stat[1] for stat in stats],
    name='Recall',
    width=0.5,
    marker=dict(
        color=colors,
        line=dict(
            color='rgb(8,48,107)',
            width=0.5),
        ),
    opacity=0.6
)

data = [trace]
layout = go.Layout(
    title='UD Path - Precision',
    xaxis_title="Relation",
    yaxis_title="Precision",        
    barmode='overlay',
    width=1200,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
        
    )
    
        
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [10]:
stats

[('per:title', 0.1548),
 ('org:alternate_names', 0.1457),
 ('org:top_members/employees', 0.1409),
 ('per:age', 0.139),
 ('per:charges', 0.0504),
 ('per:parents', 0.0484),
 ('overall', 0.0477),
 ('org:founded', 0.0452),
 ('per:cities_of_residence', 0.045),
 ('per:employee_of', 0.0415),
 ('org:city_of_headquarters', 0.0405),
 ('per:origin', 0.0385),
 ('org:founded_by', 0.0304),
 ('per:religion', 0.027),
 ('per:city_of_death', 0.0263),
 ('org:number_of_employees/members', 0.0227),
 ('per:date_of_birth', 0.0197),
 ('per:countries_of_residence', 0.019),
 ('per:children', 0.0177),
 ('org:country_of_headquarters', 0.0175),
 ('org:stateorprovince_of_headquarters', 0.0165),
 ('per:other_family', 0.0165),
 ('per:spouse', 0.0162),
 ('org:website', 0.0159),
 ('per:schools_attended', 0.0158),
 ('per:date_of_death', 0.0147),
 ('per:siblings', 0.0127),
 ('org:political/religious_affiliation', 0.0125),
 ('per:stateorprovinces_of_residence', 0.0119),
 ('per:city_of_birth', 0.0118),
 ('per:cause_of_deat